<a href="https://colab.research.google.com/github/ludovicolanni/neural-music-composer/blob/master/SequencesGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Sequences Generation

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Import the necessary objects


In [ ]:
# vocabulary: character -> index

import pickle

def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

char_indices = load_obj('/content/drive/My Drive/char_indices.pkl')


In [ ]:
# vocabulary: index -> character

chars = {v: k for k, v in char_indices.items()}
print(chars)

{0: ' ', 1: '#', 2: '-', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'e', 21: '|'}


In [ ]:
# test data

x_test = np.load('/content/drive/My Drive/RNN_data/x_test.npy')

print(x_test.shape)

(17357, 512, 22)


## Define the Sampling Function


In [ ]:
# let's define the sampling function that depends on the entropy level we assign to it
# the lower the temperature, the more predictable the results.

def sample_from_distribution(preds, temperature=1.0):

  preds = np.asarray(preds).astype('float64')
  new_preds = np.log(preds)/temperature
  exp_preds = np.exp(new_preds)
  w_preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1, w_preds, 1)
  
  return np.argmax(probas)    # returns the character corresponding to the max reweighted probability

## Define the generation loop

In [ ]:
# let's build a function for custom generation given a trained model -> keep the seed sequence

def generate_text(model, seed_set_of_sequences, chars, temp=0.01, n=1000):

  ''' temp = sampling temperature,
  n = number of characters to generate,
  model = neural model already trained,
  seed_set_of_sequences = 3D array of one-hot encoded sequences (e.g. x_test),
  chars = dictionary (vocabulary from index to char) '''
    
  import random
  import sys

  # Select a text seed at random among the 
  random_number = random.randint(0, seed_set_of_sequences.shape[0] - 1)
  seed_array = seed_set_of_sequences[random_number]

  generated_text = ''

  for i in range(seed_array.shape[0]):   # 512
    generated_text = generated_text + chars[np.argmax(seed_array[i])]

  print('--- Generating with seed: "' + generated_text + '"')

  # We generate n characters
  for i in range(n):

    if i%200==0:
      print('generating character number: ',i)
    
    preds = model.predict(seed_array.reshape(1,seed_array.shape[0],seed_array.shape[1]), verbose=0)[0]
    next_index = sample_from_distribution(preds, temp)
    next_char = chars[next_index]

    generated_text += next_char
            
    pred_array = np.zeros(shape=(seed_array.shape[1]),dtype='bool')
    pred_array[next_index] = True
    seed_array = np.vstack((seed_array,pred_array))
    seed_array = seed_array[1:]

  return generated_text    

In [ ]:
# let's build a function for custom generation given a trained model -> not keeping the seed sequence

def generate_text_2(model, seed_set_of_sequences, chars, temp=0.01, n=1000):

  ''' temp = sampling temperature,
  n = number of characters to generate,
  model = neural model already trained,
  seed_set_of_sequences = 3D array of one-hot encoded sequences (e.g. x_test),
  chars = dictionary (vocabulary from index to char) '''
    
  import random
  import sys

  # Select a text seed at random among the 
  random_number = random.randint(0, seed_set_of_sequences.shape[0] - 1)
  seed_array = seed_set_of_sequences[random_number]

  seed_text = ''
  generated_text = ''

  for i in range(seed_array.shape[0]):   # 512
    seed_text = seed_text + chars[np.argmax(seed_array[i])]

  print('--- Generating with seed: "' + seed_text + '"')

  # We generate n characters
  for i in range(n):

    if i%200==0:
      print('generating character number: ',i)
    
    preds = model.predict(seed_array.reshape(1,seed_array.shape[0],seed_array.shape[1]), verbose=0)[0]
    next_index = sample_from_distribution(preds, temp)
    next_char = chars[next_index]

    generated_text += next_char
            
    pred_array = np.zeros(shape=(seed_array.shape[1]),dtype='bool')
    pred_array[next_index] = True
    seed_array = np.vstack((seed_array,pred_array))
    seed_array = seed_array[1:]

  return generated_text 

## Generation with LSTM based model

In [ ]:
# import the trained model

from tensorflow import keras

lstm_1_100 = keras.models.load_model('/content/drive/My Drive/RNN_models/lstm_1_100.h5')

### temperature 0.01

In [ ]:
# generate with t=0.01

results_lstm_t001_n1000 = generate_text(model=lstm_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.01,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_lstm_t001_n1000[512:])

--- Generating with seed: "2|E7-88 C5-72|E5-77|G5-80|E6-92|E7-88 C5-72|E5-77|G5-85|C6-84|E6-92|E7-88 C5-72|E5-77|G5-85|C6-84|E6-92|C7-90|E7-88 C5-72|E5-77|G5-85|C6-84|E6-92|C7-90|E7-88 B5-87 e F6-74 e G6-76 C5-44|E5-62|G6-76|E7-81 C5-44|E5-62|E6-81|G6-76|E7-81 C5-49|F5-84|G#5-84|F6-90 C5-63|F5-77|G#5-62|F7-89 C5-63|F5-81|G#5-90|F6-79|G#6-85|F7-89 C5-75|F#5-92|A5-91 F#7-92 C5-72|A5-85|A6-88 e C#5-91|A#6-88 C#5-91|A#6-88 C#5-91|A#6-88|B6-82|F#7-98 C#5-91|D#5-92|C6-93|A#6-88|B6-82|F#7-98 C#5-91|D#5-92|C6-93|A#6-88|B6-82|C7-85|F#7-98 C#5"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------  -89|D#6-95 A#3-82|A#5-90|B5-91|B5-101 D#4-83|D#4-84|G#4-92|A#4-92|A#4-92|A#4-90|A5-90|B5-91|B5-101 D#4-101|G#--88|C#5-92|E5-98|G#5-92|A#5-104 C#4-83|D#5-92|E5-90|G#5-102 A#3-90|C#4-84|E4-84|G#4-88|C#5-92|E5-90|G#5-104 C#4-88|D#5-92|E5-90|G#5-104 C#4-92|A4-

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_lstm_t001_n1000.txt', 'w') as f:
    f.write(results_lstm_t001_n1000)

### temperature 0.2

In [ ]:
# generate with t=0.2

results_lstm_t02_n1000 = generate_text(model=lstm_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.2,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_lstm_t02_n1000[512:])

--- Generating with seed: "C6-78 C5-60|C6-78 F4-72 A4-65|C5-77|C6-93 C5-77|C6-93 E4-81|C5-77|C6-93 C5-77|C6-93 D4-75|C6-93 D4-75|C6-93 e e e E4-66 E4-66|A5-89 e E3-52|G#5-85 e e e C#5-42|C#6-61 G#4-55|C#5-42|C#6-61 C#5-42|C#6-61 F#4-62 e C#5-56|C#6-77 E4-67|C#5-56|C#6-77 C#5-56|C#6-77 D4-60|C#5-56|C#6-77 D4-60|C#6-77 e F#4-63|B4-65|B5-83 e E4-55 e e G#4-75|G#5-82 e e A3-68 e e e A3-87 B2-59|A3-87|D4-59|D5-80 A3-87 A3-87 C#3-62|A3-87 A3-87 A3-87|E4-55 A3-87 A3-87 D3-66|A3-87 A3-87 A3-87 E3-72|A3-87|G#4-89|G#5-90 A3-87 A3-87 A2-66|A3-8"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------  0 A3-79|A4-82|C5-81 A3-79|A4-89 A3-79|C#4-84|E4-78|G4-77|C5-81 A3-79|C#4-83|A4-87 A3-71|C#4-80|A4-87 A3-79|C#4-81|A4-87 A3-71|C#4-80|A4-87 A3-71|C#4-84|E4-75|A4-87 G3-84|F4-76|A4-87 D4-82|A4-87 D4-82|A4-84 D4-74|A4-84 A4-79 A3-69|C#4-81|E4-75 A3-76|C4-76|E

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_lstm_t02_n1000.txt', 'w') as f:
    f.write(results_lstm_t02_n1000)

### temperature 0.5

In [ ]:
# generate with t=0.5

results_lstm_t05_n1000 = generate_text(model=lstm_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.5,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_lstm_t05_n1000[512:])

--- Generating with seed: " F4-67|B4-75 e e F#4-66|A4-82 e A4-73 D#4-71 D#4-71 A4-73 C#4-56|A4-65 e F#5-63 F#5-63 B4-45|F#5-63 B4-45 A4-51 e E4-44 e G#4-69 e e A3-47|E4-59 E4-59 A4-71 G#4-80 e G#3-61|E4-67 E4-67|A4-65 E3-77 E4-76 A3-63|E4-82 A3-63|E4-82|C#5-83 E4-82|A4-84 F#3-69|G#4-88 e G#3-69|E4-76 G#3-69|E4-76|E5-83 C#5-88|D#5-90 A3-86|B4-79 e B3-87|G#4-87|A4-79 B4-83 e F#4-85 e E2-74|E4-77 E2-74|E4-77 E2-74|E4-77 E2-74|E4-77 E2-74|E4-77|E5-86 E2-74|E4-77|E5-86 E2-74|E4-77|E5-86|F5-82 E2-74|E4-77|E5-86|F5-82 E2-54|F5-82 E2-54|D5-7"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------  2 A2-57|F5-73 G2-49|D5-66 G2-47|A4-63 G4-63 G4-62 G4-60 A4-61 B4-63 A4-63 A4-65 A4-65 A2-57|B4-66 A2-69|B4-70 A2-67 G2-67 G2-61|B4-70 A2-57|C5-71 D2-57|G3-61|F#4-71 D2-59|A4-66 F2-51|A#3-51|A4-69 C3-63|D4-59|D5-65 A2-51|D3-59|D5-59 C2-59|A3-59|G4-65|C5-62 

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_lstm_t05_n1000.txt', 'w') as f:
    f.write(results_lstm_t05_n1000)

#### Generation of a set of songs with this temperature

In [ ]:
# generate 10 songs

dict_lstm_t05_n1000 = {}

for i in range(1,11):
  print('#### iteration ',i)
  dict_lstm_t05_n1000[i] = generate_text(model=lstm_1_100,
                                         seed_set_of_sequences=x_test,
                                         chars=chars,
                                         temp=0.5,
                                         n=1000)


#### iteration  1
--- Generating with seed: "-90|B6-83|C7-94|D#7-87|E7-91|F7-79|F#7-102 B4-82|C5-83|D#5-75|E5-84|F5-85|F#5-86|A5-84|A#5-82|B5-88|C6-89|D#6-83|E6-87|F6-83|F#6-86|A6-91|A#6-90|B6-83|C7-94|D#7-87|E7-91|F7-79|F#7-102 A4-68|A#4-84|B4-82|C5-83|D#5-75|E5-84|F5-85|F#5-86|A5-84|A#5-82|B5-88|C6-89|D#6-83|E6-87|F6-83|F#6-86|A6-91|A#6-90|B6-83|C7-94|D#7-87|E7-91|F7-79|F#7-102 A#4-69|B4-60|D5-11 A4-72 e e e e e e D#4-76 D4-75 C#4-76 C4-71 e e e e e e e C4-64 D#4-51 G#3-63 C4-52 D#4-57 e e e e e e e e e e D4-60 e G3-41 D#3-51|G4-74 G3-44 A3-48 e e e"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800
#### iteration  2
--- Generating with seed: "6 A#4-87|D5-75|G5-73|A#5-82 e e e A5-83|C6-83 A4-69|A5-83|C6-83 A4-63|C6-83 G4-63|C6-83 G4-63|G5-63|A#5-73 A#5-73 A#5-73 A#5-73 D#5-76|F#5-70|A5-74 A#4-88 A4-87 A#4-93|D5-73|G5-69|A#5-82 e e e G4-76|B5-81|D6-86 A4-63|B5-81 A4-64

In [ ]:
# save the songs

for i in dict_lstm_t05_n1000:
  
  path = '/content/drive/My Drive/RNN_generatedtext/results_lstm_t05_n1000_{}.txt'.format(i)

  with open(path, 'w') as f:
    f.write(dict_lstm_t05_n1000[i])

In [ ]:
# generate 10 songs without keeping the seed sequence (1500 characters)

dict_lstm_t05_n1000 = {}

for i in range(11,21):
  print('#### iteration ',i)
  dict_lstm_t05_n1000[i] = generate_text_2(model=lstm_1_100,
                                         seed_set_of_sequences=x_test,
                                         chars=chars,
                                         temp=0.5,
                                         n=1500)


#### iteration  11
--- Generating with seed: "|B5-80 B4-57|D#5-60 B4-57|D#5-60 e e e G5-54|B5-72 e e e e e C5-50 G#5-65|C6-79 e e e F4-65|G#4-69 F#4-71|A4-77 C5-62|D#5-69 e e G#5-76|C#6-92 C#5-72|F5-67 F4-76|G#4-67|C#5-72|F5-67 e e e A5-82|C6-79 C#6-96 e e e A5-75|C6-66 C#5-67|F5-66 C#5-67|F5-66 e A5-63|C#6-80 C#5-54|F5-64 C#5-54|F5-64 e A4-58|C#5-68 e e e e e E4-42|A4-56|C#5-52|C#6-63 C#7-76 C#6-72|C#7-76 C#6-72|C#7-76 C#6-72|C#7-76 E4-43|A4-80 G#5-69 A5-73 C#6-73 A5-67 e F#6-72|B6-76 e e F#6-79 F#6-79 D#5-70|F#6-79 F#5-69 A5-67 E4-63|A4-63|C#5-66 C#5"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800
generating character number:  1000
generating character number:  1200
generating character number:  1400
#### iteration  12
--- Generating with seed: "A5-92 D3-61|E5-91 e e e e A2-50 e e e E5-66 E5-66 E5-66 E2-48|E4-49|E5-66 E2-48|E4-49|E5-66 E2-48|E4-49|D5-56 E2-48|C#5-6

In [ ]:
# save the songs

for i in dict_lstm_t05_n1000:
  
  path = '/content/drive/My Drive/RNN_generatedtext/results_lstm_t05_n1000_{}.txt'.format(i)

  with open(path, 'w') as f:
    f.write(dict_lstm_t05_n1000[i])

### temperature 0.8

In [ ]:
# generate with t=0.8

results_lstm_t08_n1000 = generate_text(model=lstm_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.8,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_lstm_t08_n1000[512:])

--- Generating with seed: "74 E3-38|A3-46|C4-45|C5-74 A3-46 e G#4-74 A2-55|A4-68 e e e E5-61|A5-58 E5-61 E5-61 E5-61 E2-51|E4-51|E5-61 E2-51|E4-51|E5-61 E2-51|E4-51|D5-58 E2-51|E4-51 E2-51 e A4-60 A3-54|D5-75 e E5-77 E5-77 E5-77 E2-58|E4-61|E5-77 E2-58|E4-61|E5-77 E2-58|E4-61|D5-65 E2-58 E2-58|D5-76 A2-46|C#5-81 A4-70 D5-87 e E5-88 E5-88 E2-69|E4-75|E5-88 E2-69|E4-75|E5-88 E4-75|E5-88 E4-75|C#5-77|D5-76|E5-88 E4-75|B4-83|C#5-77|D5-76|E5-88 D5-80 A4-78 C#5-86 e A5-92 D3-61|E5-91 e e e e A2-50 e e e E5-66 E5-66 E5-66 E2-48|E4-49|E5-66 "
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------  B2-55|E5-55 C#4-56|E5-66 F#4-50|E5-66 G3-71|G#4-68 e e e e e C5-68|E5-67 A4-65 G4-67 A4-62 A4-75 A4-70 C#4-72|E4-60|F#4-70 G4-71 e C4-74|E4-81 e e F#4-70 A4-78 D5-78 A3-66|A4-71 B3-64|A4-70 D3-63|A4-67 G3-60|D5-78 D3-57|F#3-69|B4-71 F3-52|A4-60 G2-64 C3-69

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_lstm_t08_n1000.txt', 'w') as f:
    f.write(results_lstm_t08_n1000)

## Generation with GRU based model

In [ ]:
# import the trained model

from tensorflow import keras

gru_1_100 = keras.models.load_model('/content/drive/My Drive/RNN_models/gru_1_100.h5')

### temperature 0.01

In [ ]:
# generate with t=0.01

results_gru_t001_n1000 = generate_text(model=gru_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.01,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_gru_t001_n1000[512:])

--- Generating with seed: "3-79|D#4-81|F4-75|A#4-74|B4-88|C#5-78|F#5-79|G#5-73 D#3-82|F3-84|F#3-59|A#3-75|B3-79|D#4-81|F4-75|A#4-74|B4-88|C#5-78|F#5-79|G#5-73|B5-81 D#3-82|F3-84|F#3-59|A#3-75|B3-79|D#4-81|F4-75|A#4-74|B4-88|C#5-78|F#5-79|G#5-73|B5-81 A2-72|A#2-41|D#3-82|F3-84|F#3-59|A#3-75|B3-79|D#4-81|F4-75|A#4-74|B4-88|C#5-78|F#5-79|G#5-73|B5-81|F#6-86 F#2-79|A2-72|A#2-41|D#3-82|F3-84|F#3-59|A#3-75|B3-79|D#4-81|F4-75|A#4-74|B4-88|C#5-78|F#5-79|G#5-73|B5-81|F#6-86|A#6-81 e e e e e F7-101 F#1-71|F#2-94|F#6-96|F#7-118 e e e e e e e e "
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------  e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e 

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_gru_t001_n1000.txt', 'w') as f:
    f.write(results_gru_t001_n1000)

### temperature 0.2

In [ ]:
# generate with t=0.2

results_gru_t02_n1000 = generate_text(model=gru_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.2,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_gru_t02_n1000[512:])

--- Generating with seed: "|B5-83 e E4-55 e e G#4-75|G#5-82 e e A3-68 e e e A3-87 B2-59|A3-87|D4-59|D5-80 A3-87 A3-87 C#3-62|A3-87 A3-87 A3-87|E4-55 A3-87 A3-87 D3-66|A3-87 A3-87 A3-87 E3-72|A3-87|G#4-89|G#5-90 A3-87 A3-87 A2-66|A3-87|E4-78|E5-89 A3-87 A3-87 A3-87 A3-87 A3-87 A3-87 A3-87 C#3-65|A3-87|A4-81|A5-89 A3-87 C#3-71|A3-87|E5-91 D3-82|F#4-86|D5-92 e E3-87|E4-83|C#5-87 E3-87 E3-87 E2-83|E3-87|D4-72|B4-80 e A2-84 A2-84 A2-84 E4-77|B4-90 e A2-86 A2-86|C#4-81|A4-83 A2-86 B4-90 e A2-59|C#4-67|A4-79 E4-74|B4-83 e A2-73|C#4-73|A4-82"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------   D#3-71|A4-83 D#3-76|A#4-89 D#3-76|A#4-89 A#2-76|A#4-89 A#2-79|A#4-81 D#3-77|F#4-71 A#2-78|A#3-78|D#4-78 A#2-76|F#3-78|A#3-78|A4-77 F#2-77|F#3-78|A4-73 D#3-71|F#4-78 A#2-70|F#4-76 F#2-67|F#3-63|D#4-74 A#2-70|F#3-78|A#3-73|F#4-78 A#2-77|F#3-76|A#3-77|D#4-78

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_gru_t02_n1000.txt', 'w') as f:
    f.write(results_gru_t02_n1000)

### temperature 0.5

In [ ]:
# generate with t=0.5

results_gru_t05_n1000 = generate_text(model=gru_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.5,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_gru_t05_n1000[512:])

--- Generating with seed: "87|B5-80|C#6-85|F6-88|F#6-76|G#6-81|A#6-83|B6-102|C7-79|C#7-83|D#7-77|F7-90 B4-81|D#5-80|F5-75|F#5-79|G#5-76|A#5-87|B5-91|C#6-85|F6-88|F#6-76|G#6-81|A#6-83|B6-102|C7-79|C#7-83|D#7-77|F7-90 B4-85|D#5-80|F5-75|F#5-79|G#5-79|A#5-87|B5-91|C#6-85|F6-88|F#6-76|G#6-81|A#6-83|B6-102|C7-79|C#7-83|D#7-77|F7-90 G#4-72|B4-85|D#5-80|F5-78|F#5-79|G#5-79|A#5-87|B5-91|C#6-85|F6-88|F#6-76|G#6-81|A#6-83|B6-102|C7-79|C#7-83|D#7-77|F7-90 G#4-72|A#4-83|B4-85|D#5-80|F5-78|F#5-79|G#5-84|A#5-87|B5-91|C#6-85|F6-88|F#6-76|G#6-81|A#6"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------  -80|G#6-87|E6-89 F#3-70|A#3-74|D#5-80|C#6-93|A#6-97 F3-72|A#3-74|D#5-82|D#6-93|D6-89|G#6-87 F#3-78|D#3-72|G#3-78|D#5-73|D#6-83|D#6-87 F#3-79|D#3-74|D#5-73|D#6-93 D#3-72|D#3-70|F#3-74|A#3-73|D#5-77|F#5-73|D#6-91|D#6-83 A3-76|A#3-77|A#3-74|F#5-77|D#6-81|F#6-

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_gru_t05_n1000.txt', 'w') as f:
    f.write(results_gru_t05_n1000)

#### Generation of a set of songs with this temperature

In [ ]:
# generate 10 songs

dict_gru_t05_n1000 = {}

for i in range(1,11):
  print('#### iteration ',i)
  dict_gru_t05_n1000[i] = generate_text(model=gru_1_100,
                                         seed_set_of_sequences=x_test,
                                         chars=chars,
                                         temp=0.5,
                                         n=1000)


#### iteration  1
--- Generating with seed: "81 e e e e e e e e e e e e e e e e e e e e A#5-87 e e e e e e e e e e e C#2-62 C#2-62 e F#2-56 F#2-55 D2-53 D2-49 D2-47 B1-51 B1-55 e D2-66 e e e e e e D6-41 A6-67 e e e B1-52|D6-62 F#1-51|A#6-76|D7-73 F#1-57|F#6-72 F#1-52 F#1-57|A#5-59|D6-75 F#1-53|F#5-67 F#1-63|D5-70 F#1-49|A#5-60|C#6-79 F#5-68 F#1-60|C#5-72 F#1-51|F#5-59|A#5-77 C#5-69 A#4-74 F#1-68|A#5-75|D6-80 F#1-61|F#5-77 F#1-58|D5-67 F#1-49|A#4-57|D5-70 F#1-55|F#4-63 D4-68 F#1-57|A#4-58|C#5-82 F#4-64 F#1-44|C#4-67 F#1-54|F#4-63|A#4-72 C#4-67 F#1-65 F"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800
#### iteration  2
--- Generating with seed: "-91|F7-79|F#7-102 A6-91|A#6-90|B6-83|C7-94|D#7-87|E7-91|F7-79|F#7-102 F6-81|A6-91|A#6-90|B6-83|C7-94|D#7-87|E7-91|F7-79|F#7-102 D#6-83|F6-81|F#6-86|A6-91|A#6-90|B6-83|C7-94|D#7-87|E7-91|F7-79|F#7-102 D#6-83|E6-87|F6-83|F#6-86|A6

In [ ]:
# save the songs

for i in dict_gru_t05_n1000:
  
  path = '/content/drive/My Drive/RNN_generatedtext/results_gru_t05_n1000_{}.txt'.format(i)

  with open(path, 'w') as f:
    f.write(dict_gru_t05_n1000[i])

In [ ]:
# generate 10 songs without keeping the seed sequence (1500 characters generated)

dict_gru_t05_n1000 = {}

for i in range(11,21):
  print('#### iteration ',i)
  dict_gru_t05_n1000[i] = generate_text_2(model=gru_1_100,
                                         seed_set_of_sequences=x_test,
                                         chars=chars,
                                         temp=0.5,
                                         n=1500)


#### iteration  11
--- Generating with seed: "F6-81|G6-81|G#6-81|A#6-101|B6-88|C7-77|C#7-55|D#7-82|F7-85 G#4-70|B4-78|F5-83|E6-79|F6-81|G6-81|G#6-82|A#6-101|B6-88|C7-77|C#7-55|D#7-82|F7-85 C#4-81|F#6-89 F#4-68|A#6-86|D#7-84 A#4-78|C#6-82|C#7-58 C#5-76|F#6-78|B6-82 F#5-81|A#5-88 A#5-77|C#6-80 A5-87|F#6-71 F#5-84|D#6-41 C5-77|F#5-84|D#6-85|F#6-81 e G#6-68 G#6-68|D#7-77|F7-90 G#6-68|C7-79|C#7-83|D#7-77|F7-90 G#6-68|A#6-79|C7-79|C#7-83|D#7-77|F7-90 G#6-68|A#6-79|B6-89|C7-79|C#7-83|D#7-77|F7-90 G#6-75|A#6-79|B6-89|C7-79|C#7-83|D#7-77|F7-90 C#6-80|G#6-75|A#6"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800
generating character number:  1000
generating character number:  1200
generating character number:  1400
#### iteration  12
--- Generating with seed: "-73 A4-60|E5-73 e G5-97 e e e A4-67 e F#5-78 F#5-78 F#5-81 F#5-81 F#5-81 E5-71|F#5-81 E5-71|F#5-81 E5-71|F#5-81 D3-36 D3-

In [ ]:
# save the songs

for i in dict_gru_t05_n1000:
  
  path = '/content/drive/My Drive/RNN_generatedtext/results_gru_t05_n1000_{}.txt'.format(i)

  with open(path, 'w') as f:
    f.write(dict_gru_t05_n1000[i])

### temperature 0.8

In [ ]:
# generate with t=0.8

results_gru_t08_n1000 = generate_text(model=gru_1_100,
                                       seed_set_of_sequences=x_test,
                                       chars=chars,
                                       temp=0.8,
                                       n=1000)

print()
print('newly generated text: ---------- ',results_gru_t08_n1000[512:])

--- Generating with seed: "5 e F#3-63 A3-60 C#4-62 D4-70 A#3-58 e D#3-60|C4-66 G3-54 A3-57 A#3-66|D4-74 G3-47 e A#2-58 G3-51 D#3-50 D3-69 C#3-56 D3-58 D#3-62|G3-47|C4-71 e e A3-64 e F#2-54 D2-51 D2-44 D2-52 D2-47 D2-53 D2-57 D2-53|F#3-66|A#3-65 D2-58|F#3-74|A#3-74 D2-58|A#3-76 D2-64|D#3-70|G3-79 D2-56 D2-55 e e e e D2-60 e e e e e e e e e e e e e e e e e e e e e e e D2-47 e e e e e e e e e e e D2-54 e D#2-63 D#2-56 D#2-55 D2-61 D2-51 D2-49 C2-52|D#4-53|F#4-53|A4-68 C2-50 C2-55 e e D2-59 e D#2-50 e e e e e e D2-62 e e F#5-64|C6-74 D#2"
generating character number:  0
generating character number:  200
generating character number:  400
generating character number:  600
generating character number:  800

newly generated text: ----------  -45|C6-77 F#3-51|F#5-68|D#5-65|D#5-66 C#3-69|D5-81 D#3-62|F#4-74|D5-82 D#5-82 A#3-72|C5-71 F#3-68|D#4-60|A4-74 C#4-68|D4-64|A4-74|C#5-83 D3-72|F#4-70|C#5-84 C#3-71|F#4-73|E4-74|F#4-71|G#4-82|D5-73 A#2-73 G#2-73|A#3-72|E3-75|A#4-88 C#4-79|D#4-76|D#4-82|E4-7

In [ ]:
# save the generated text

with open('/content/drive/My Drive/RNN_generatedtext/results_gru_t08_n1000.txt', 'w') as f:
    f.write(results_gru_t08_n1000)